Unit tests for the [Epanechnikov distribution](https://en.wikipedia.org/wiki/Epanechnikov_distribution).

In [ ]:
import numpy as np

def pdf(x, c):
  return 3/(4*c) * np.maximum(0, 1 - (x/c)**2)

def cdf(x, c):
  xc = np.clip(x, -c, c)
  return 1/2 + 3/(4*c) * xc - xc**3 / (4*c**3)

cs = 2.**np.linspace(-5, 5, 11)

def test_support():
  # Assert that the support of the distribution is [-c, c]
  for c in cs:
    assert pdf(np.nextafter(-c, -np.inf), c) == 0
    assert pdf(np.nextafter(-c, np.inf), c) > 0
    assert pdf(np.nextafter(c, -np.inf), c) > 0
    assert pdf(np.nextafter(c, np.inf), c) == 0
    assert np.all(pdf(np.linspace(np.nextafter(-c, np.inf), np.nextafter(c, -np.inf), 1001), c) > 0)
    assert np.all(pdf(np.linspace(-1e80, np.nextafter(-c, -np.inf), 1001), c) == 0)
    assert np.all(pdf(np.linspace(np.nextafter(c, np.inf), 1e80, 1001), c) == 0)

def test_normalized():
  # Assert that the PDF integrates to 1.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 10001)
    p = pdf(x, c)
    np.testing.assert_allclose(np.sum(p) * (x[1] - x[0]), 1)

def test_cdf():
  # Assert that differentiating the CDF yields the PDF.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 100001)
    np.testing.assert_allclose(pdf((x[:-1] + x[1:])/2, c), np.diff(cdf(x, c)) / (x[1] - x[0]), atol=1e-5)

def test_mean():
  # Assert that the sample mean is zero.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 10001)
    p = pdf(x, c)
    w = p / np.sum(p)
    np.testing.assert_allclose(np.sum(w * x), 0., atol=1e-7)

def test_variance():
  # Assert that the sample variance is c**2 / 5.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 1000001)
    p = pdf(x, c)
    w = p / np.sum(p)
    np.testing.assert_allclose(np.sum(w * x**2), c**2 / 5)

def test_median():
  # Assert that the median is 0.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 1000001)
    np.testing.assert_allclose(cdf(0, c), 0.5)

def test_mode():
  # Assert that the mode is 0.
  for c in cs:
    x = np.linspace(-2*c, 2*c, 1000001)
    p = pdf(x, c)
    is_max = p == np.max(p)
    np.testing.assert_allclose(np.sum(x * is_max) / np.sum(is_max), 0)

# Super simple test harness.
if __name__ == '__main__':
  for t in [f for n, f in globals().items() if n.startswith('test_')]:
    n = t.__name__
    try:
      t()
      print(f"✓ {n} passed")
    except AssertionError as e:
      print(f"✗ {n} failed: {str(e)}")
    except Exception as e:
      print(f"✗ {n} error: {str(e)}")

✓ test_support passed
✓ test_normalized passed
✓ test_cdf passed
✓ test_mean passed
✓ test_variance passed
✓ test_median passed
✓ test_mode passed
